### Load the trainned prunned model

In [ ]:
from qkeras.utils import _add_supported_quantized_objects
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow_model_optimization.sparsity.keras import strip_pruning

In [ ]:
import math
from collections import defaultdict

def reuse_percentage_to_factors(model, serial_pct: float = 1.0):
    """
    Map each Dense / Conv / DepthwiseConv layer to a valid hls4ml ReuseFactor.

    Parameters
    ----------
    model       : Keras / QKeras model
    serial_pct  : float in [0, 1]
        1.0 → completely serial (largest legal reuse factor)
        0.0 → completely parallel (ReuseFactor = 1)

    Returns
    -------
    dict {layer_name: reuse_factor}
    """
    from tensorflow.keras.layers import Dense, Conv2D, DepthwiseConv2D
    from qkeras import QDense, QConv2D, QDepthwiseConv2D

    if not 0.0 <= serial_pct <= 1.0:
        raise ValueError("serial_pct must be in [0, 1]")

    reuse = {}

    
    def legal_divisor(n, candidate):
        """Decrease candidate until it cleanly divides n."""
        while candidate > 1 and n % candidate:
            candidate -= 1
        return max(1, candidate)
    

    for layer in model.layers:
        if isinstance(layer, (Dense, QDense)):
            total = layer.input_shape[-1] * layer.units          # Nin × Nout

        elif isinstance(layer, (Conv2D, QConv2D)):
            kh, kw   = layer.kernel_size
            cin      = layer.input_shape[-1]
            cout     = layer.filters
            total    = kh * kw * cin * cout                      # per-pixel MACs

        elif isinstance(layer, (DepthwiseConv2D, QDepthwiseConv2D)):
            kh, kw   = layer.kernel_size
            cin      = layer.input_shape[-1]
            total    = kh * kw * cin                             # per-pixel MACs

        else:
            continue  # no MACs → nothing to tune

        #   serial_pct = 1.0  → target_rf = total  (most serial)
        #   serial_pct = 0.0  → target_rf = 1      (fully parallel)
        target_rf = int(round(total * serial_pct))
        target_rf = min(max(1, target_rf), total)

        # Snap downward to the nearest divisor so that total % rf == 0
        rf = legal_divisor(total, target_rf)

        reuse[layer.name] = rf
        print(f"{layer.name:20s}  MACs={total:6d}  ReuseFactor={rf}")

    return reuse


In [ ]:
import math
from collections import defaultdict

def reuse_factors_with_serial_pct(model):
    """
    For each Dense / Conv / DepthwiseConv layer in a model, print all valid reuse factors 
    and their corresponding serial_pct (ReuseFactor / total MACs).

    Parameters
    ----------
    model : Keras / QKeras model
    """
    from tensorflow.keras.layers import Dense, Conv2D, DepthwiseConv2D
    from qkeras import QDense, QConv2D, QDepthwiseConv2D

    def get_total_macs(layer):
        if isinstance(layer, (Dense, QDense)):
            return layer.input_shape[-1] * layer.units
        elif isinstance(layer, (Conv2D, QConv2D)):
            kh, kw = layer.kernel_size
            cin = layer.input_shape[-1]
            cout = layer.filters
            return kh * kw * cin * cout
        elif isinstance(layer, (DepthwiseConv2D, QDepthwiseConv2D)):
            kh, kw = layer.kernel_size
            cin = layer.input_shape[-1]
            return kh * kw * cin
        else:
            return None  # Unsupported layer

    def get_divisors(n):
        """Return all positive integers that divide n."""
        return sorted({i for i in range(1, n + 1) if n % i == 0})

    for layer in model.layers:
        total_macs = get_total_macs(layer)
        if total_macs is None:
            continue

        print(f"\nLayer: {layer.name} ({layer.__class__.__name__}) - Total MACs: {total_macs}")
        print(f"{'ReuseFactor':>12} | {'SerialPct':>10}")
        print("-" * 27)

        for rf in get_divisors(total_macs):
            serial_pct = rf / total_macs
            print(f"{rf:12d} | {serial_pct:10.4f}")


In [ ]:



model_dir = 'models'
model_path = f'{model_dir}/quantized_pruned_cnn_model.h5'
co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude


qmodel = tf.keras.models.load_model(model_path, custom_objects=co)
qmodel = strip_pruning(qmodel)

In [ ]:
qmodel.summary()

In [ ]:
reuse_factors_with_serial_pct(qmodel)

In [ ]:
project_folder = 'Projects_Reuse_Factor_Analysis'
project_path = f'{project_folder}/Packing'
backend = 'Vitis'
default_precision = 'ap_fixed<16,6>'
part = 'xczu5ev-sfvc784-1-i'
project_name = 'packing_impl'

In [ ]:
import hls4ml
import utils.plotting as plotting

# Generate base HLS config
hls_config = hls4ml.utils.config_from_keras_model(
    qmodel,
    granularity='name',
    backend=backend,
    default_precision=default_precision
)

# Set model-level precision
hls_config['Model']['Precision'] = default_precision

# Force Resource strategy globally
for lname, lcfg in hls_config['LayerName'].items():
    lcfg['Strategy'] = 'Resource'

# Inject computed reuse factors
reuse_factors = reuse_percentage_to_factors(qmodel, serial_pct=0.0)
for lname, factor in reuse_factors.items():
    if lname in hls_config['LayerName']:
        hls_config['LayerName'][lname]['ReuseFactor'] = factor
    else:
        print(f"Warning: Layer {lname} not found in HLS config")

plotting.print_dict(hls_config)


# Convert and compile
hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel,
    hls_config=hls_config,
    backend=backend,
    output_dir=project_path,
    part=part,
    io_type='io_stream',
    clock_period=5,
    trace=True,
    project_name=project_name,
)
hls_model.compile()


In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
from template_injector import TemplateInjector

injector = TemplateInjector(template_dir="templates")

injector.inject(
    project_dir=project_path,
    project_name=project_name,
    force=True,  # or False to skip existing files
    packed=True
)

In [ ]:
import os
pwd = os.getcwd()
print(f"Current working directory: {pwd}")
build = True

In [ ]:
if build:
    os.system(f"cd {pwd}/{project_path} && python build_tb.py && vitis-run --mode hls --tcl build_prj.tcl && python compute_performance.py && python golden_preds.py")